## Vector to Raster: A step-by-step breakdown

This notebook processes a single line vector into a raster file with the 'speed' attribute used as the raster pixel value. </br> In our routine, we use this notebook to convert the final processed road rasters into speed surfaces we can combine with the walking friction surface in the next notebook.

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'

import pprint
from pprint import pprint

import common_rasterio_ops as rast_ops

import numpy as np

import rasterio
from rasterio import features
from rasterio import transform
from rasterio.transform import Affine
from rasterio.mask import mask
from rasterio.io import MemoryFile

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape

import json

File paths

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters that will eventually be passed from the UI
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables that will come from UI so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_lc_folder = data_loaded['local_lc_folder']
local_roads_folder = data_loaded['local_roads_folder']
local_dem_folder = data_loaded['local_dem_folder']
local_boundaries_folder = data_loaded['local_boundaries_folder']
fric_dir = data_loaded['fric_dir']
fric_dir_onroad = data_loaded['fric_dir_onroad']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']
buffer_m = data_loaded['buffer_m']
level = data_loaded['level']
if level != 'custom':
    shapefile_adm_field = data_loaded['shapefile_adm_field']
    adm_name = data_loaded['adm_name']


In [ ]:
seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".tif")])

In [ ]:
for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".tif","")   

Load Shapefile of aoi to clip the final data

In [ ]:
aoi_pth = local_boundaries_folder + level + '\\'

aoi_file = sorted([os.path.join(aoi_pth,file) \
            for file \
            in os.listdir(aoi_pth) \
            if file.endswith(".shp")])

aoi_file = aoi_file[0]

aoi = gpd.read_file(aoi_file)

aoi = aoi[aoi[shapefile_adm_field] == adm_name]
aoi = aoi.to_crs(dest_crs)

# Buffer the polygon so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(buffer_m)

### Load reference raster

Load in a raster we are aligning with (generally the Off-Road / walking raster itself) so we can match up exactly to its grid and cell size

In [ ]:
# choose the raster with the appropriate resolution
with rasterio.open(
        os.path.join(fric_dir,seasons[0]+'_walk.tif'), 'r') as src:
    base_array = src.read(1)
    base_idx = src.index
    base_profile = src.meta.copy()
    base_tform = src.transform

In [ ]:
### Exporting

export_profile = {
    "driver": "GTiff",
    "dtype": "float32",
    "crs":{'init':dest_crs},
    "height": base_array.shape[0],
    "width": base_array.shape[1],
    "count":1,
    "nodata":0,
    "transform": base_tform,
    "compress":'LZW'
}

In [ ]:
# load in the vertical distance raster
# This layer represents the extra total distance that must be traversed due to vertical gains, accounting for the likelihood of walkers using switchbacks on slopes above 30 degrees
with rasterio.open(os.path.join(local_dem_folder,'vert_dist_switchback.tif'),'r') as vertdist_switchback_src:
    vert_dist_switchback = vertdist_switchback_src.read()

### Load in road file

In [ ]:
rd_pth = local_roads_folder + 'final_roads.gpkg'

# replace with master transport file prepared in Step 1
rd = gpd.read_file(rd_pth,driver="GPKG")

# clipping the roads by the extent of your reference raster

from shapely.geometry import Polygon, box
bbox = box(*src.bounds)

rds_clip = gpd.clip(rd,bbox)

In [ ]:
for season_num in range(0, len(seasons)):
    
    current_season = seasons[season_num]
    
    # replace with speed column you're using
    speed_column = current_season + '_speed'

    # Reorder the datafram least to highest, so that the generator and therefore speed values are written similarly, 
    # This has the ultimate consequence that higher values overwrite lower values where they overlap

    rds_clip = rds_clip.sort_values(by=speed_column)

    if rds_clip.crs.is_projected == True:
        None
    else:
        rds_clip = rds_clip.to_crs(dest_crs)

#     ### Rasterizing vectors

#     Run a function to generate all the necessary parameters for our raster outputs based on the extent of the input shapefile

    bds, ht, wth, src_tform, dst_tform, shapes, scale_factor_x, scale_factor_y = rast_ops.get_raster_params(rds_clip,src,speed_column)

    # New code downsamples the code while rasterizing, saving us major headaches with resampling in memory.

    road_rast = features.rasterize(shapes,\
                      out_shape = (base_array.shape[0],\
                                   base_array.shape[1]),\
                      transform=base_tform,
                      all_touched=True,
                      dtype = np.float32)

    # check values are of the right magnitude
    road_rast[road_rast > 0]

    # Export masked version
    road_rast_mask, road_rast_mask_tform = rast_ops.clip_in_memory(road_rast,export_profile, aoi.geometry)
    
    # Convert speed surfaces to friction surfaces
    # refactor driving speeds to friction values in units of cell size / hour (e.g. 30m / hour)
    # to get values in minutes multiply by 60
    friction_drive_step1 = (1 / road_rast_mask) / (1000 / base_tform.a)  
    del road_rast_mask

    # Multiply these friction surfaces by the vertical+horizontal distance
    # we use the simple vertical distance, without switchbacks, on the assumption road switchbacks are incorporated into the vector line objects that have been rasterized
    friction_drive_step2 = np.multiply(friction_drive_step1,vert_dist_switchback) 
    del friction_drive_step1

    ### Remove index band
    friction_drive_step2 = friction_drive_step2[0, :, :]
    
    with rasterio.open(
            os.path.join(fric_dir_onroad,current_season + '_onroad.tif'), 'w',**export_profile) as dst:
        dst.write(friction_drive_step2,indexes=1)